In [10]:
import os
import pandas as pd
import requests
import googlemaps
import json
import time
from deep_translator import GoogleTranslator
from pytz import timezone
import gpxpy
from datetime import datetime, timedelta
import pytz
from sklearn.cluster import DBSCAN
import numpy as np
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import folium

In [11]:
def get_osm_address(lat, lng):
    """Récupère l'adresse depuis OpenStreetMap."""
    try:
        url = f"https://nominatim.openstreetmap.org/reverse?format=json&lat={lat}&lon={lng}"
        headers = {'User-Agent': 'TrainingLocPrivacy/1.0 ()'}
        response = requests.get(url, headers=headers)  # Timeout ajouté pour éviter les blocages
        if response.status_code == 200 and response.json().get('display_name'):
            data = response.json()
            name = GoogleTranslator(source='auto', target='en').translate(data.get('display_name', ''))
            address = GoogleTranslator(source='auto', target='en').translate(data.get('display_name', ''))
          #  print(data)
            return {
                'name': name,
                'address': address,
                'lat': data.get('lat'),
                'lon': data.get('lon'),
                'Coordinates': data.get('boundingbox'),
                'Categories': '[ ' + data.get('class', 'unknown') + ' ; ' +
                              data.get('type', 'unknown') + ' ; ' +
                              data.get('addresstype', 'unknown') + ']',
            }
    except Exception as e:
        print(f"Error retrieving the address for ({lat}, {lng}): {e}")
    return None

In [12]:
def get_osm_address_gowalla(lat, lng):
    """Récupère l'adresse et la catégorie d'un lieu depuis OpenStreetMap."""
    try:
        url = f"https://nominatim.openstreetmap.org/reverse?format=json&lat={lat}&lon={lng}"
        headers = {'User-Agent': 'TrainingLocPrivacy/1.0 (farid.yessoufou0@gmail.com)'}
        response = requests.get(url, headers=headers, timeout=5)  # Timeout pour éviter les blocages

        if response.status_code == 200 and response.json().get('display_name'):
            data = response.json()
            
            # Traduction en anglais
            name = GoogleTranslator(source='auto', target='en').translate(data.get('display_name', ''))
            address = GoogleTranslator(source='auto', target='en').translate(data.get('display_name', ''))

            # Construction de la catégorie
            categories = f"[ {data.get('class', 'unknown')} ; {data.get('type', 'unknown')} ; {data.get('addresstype', 'unknown')} ]"

            return {"name": name, "address": address, "categories": categories}
    except Exception as e:
        print(f"⚠️ Erreur lors de la récupération des métadonnées pour ({lat}, {lng}): {e}")
    return None

In [13]:
def process_and_enrich_files(directory, timer=30):
    """
    Browses the .plt files in a directory, enriches the data with OpenStreetMap
    """
    try:
        # Liste des fichiers .plt
        noms_de_fichiers = [f for f in os.listdir(directory) if f.endswith('.plt')]
        save_file_directory = os.path.join(directory, 'MetaData')
        log_file_directory = os.path.join(directory, 'Logs')

        # Créer les répertoires de sauvegarde si nécessaires
        if not os.path.exists(save_file_directory):
            os.makedirs(save_file_directory)
        if not os.path.exists(log_file_directory):
            os.makedirs(log_file_directory)

        for nom_de_fichier in noms_de_fichiers:
            file_path = os.path.join(directory, nom_de_fichier)
            enriched_data = []

            # Commencer la mesure du temps
            start_time = time.time()

            # Lire le fichier .plt et enrichir les données
            with open(file_path, 'r') as f:
                lines = f.readlines()

                # Ignorer les en-têtes (6 premières lignes)
                for line in lines[6:]:
                    try:
                        values = line.strip().split(',')
                        lat, lng = float(values[0]), float(values[1])
                        timestamp_str = f"{values[5]} {values[6]}"  # Date + Heure
                        timestamp = datetime.strptime(timestamp_str, "%Y-%m-%d %H:%M:%S").isoformat()

                        # Récupérer les données enrichies depuis OpenStreetMap
                        metadata = get_osm_address(lat, lng)
                        if metadata:
                            # Ajouter uniquement si différent du précédent
                            metadata["timestamp"] = timestamp
                           # if not enriched_data or enriched_data[-1]['address'] != metadata['address']:
                            enriched_data.append(metadata)
                    except Exception as e:
                        print(f"Error when processing the line '{line}': {e}")
            
            # Sauvegarder les résultats dans un fichier JSON
            target_file = os.path.join(save_file_directory, f"enriched_{nom_de_fichier}.json")
            with open(target_file, 'w') as f:
                json.dump(enriched_data, f, indent=4)
            
            # Calculer le temps d'exécution
            execution_time = time.time() - start_time
            log_file = os.path.join(log_file_directory, f"log_{nom_de_fichier}.txt")

            # Écrire le journal d'exécution
            with open(log_file, 'w') as log:
                log.write(f"Processed file: {nom_de_fichier}\n")
                log.write(f"Execution time: {execution_time:.2f} seconds\n")
                log.write(f"Number of enriched lines: {len(enriched_data)}\n")
                log.write(f"Result saved in: {target_file}\n")
            
            print(f"File processed  : {nom_de_fichier}, saved as : {target_file}, execution time : {execution_time:.2f} secondes")
    
    except Exception as e:
        print(f"Error in the main function  : {e}")

In [14]:
def process_all_folders():
    root_directory = 'Geolife/Data'
    
    for i in range(182):  # De 000 à 181
        folder_name = f"{i:03d}"  # Convertit 0 en '000', 1 en '001', ..., 181 en '181'
        full_path = os.path.join(root_directory, folder_name, 'Trajectory')
        
        if os.path.exists(full_path):  # Vérifier si le répertoire existe
            process_and_enrich_files(full_path)
        else:
            print(f"The directory {full_path} doesn't exist, it's ignored.")

# Appel de la fonction principale
#process_all_folders()

In [15]:
def get_last_processed_index(output_file):
    """Récupère le dernier index traité en lisant la dernière ligne du fichier de sortie."""
    if not os.path.exists(output_file) or os.stat(output_file).st_size == 0:
        return -1  # Aucun traitement encore effectué
    try:
        last_row = pd.read_csv(output_file).iloc[-1]
        return last_row.name  # Récupérer l'index de la dernière ligne
    except Exception:
        return -1

In [16]:
def process_gowalla_file(input_file, output_file):
    """Lit un fichier Gowalla et récupère les métadonnées des lieux en conservant les colonnes d'origine."""
    print("📂 Lecture du fichier Gowalla...")
    
    # Charger les données
    columns = ["user", "check-in time", "latitude", "longitude", "location id"]
    df = pd.read_csv(input_file, sep="\t", names=columns, engine="python")
    
    # Initialisation des nouvelles colonnes
    df["display_name"] = None
    df["address"] = None
    df["category"] = None
    
    # Lire le dernier index traité
    last_index = get_last_processed_index(output_file)
    print(f"🔄 Reprise à partir de l'index {last_index+1}")
    
    # Ouvrir le fichier CSV en mode écriture et écrire les en-têtes si nécessaire
    if last_index == -1:
        df.iloc[:0].to_csv(output_file, index=False, encoding="utf-8", mode='w')
    
    for index, row in df.iterrows():
        if index <= last_index:
            continue  # Passer les lignes déjà traitées
        
        lat, lng = row["latitude"], row["longitude"]
        
        # Récupération des métadonnées
        metadata = get_osm_address_gowalla(lat, lng)
        print(metadata)
        if metadata:
            df.at[index, "display_name"] = metadata.get("name", "")
            df.at[index, "address"] = metadata.get("address", "")
            df.at[index, "category"] = metadata.get("categories", "")
        
        # Sauvegarde incrémentale de la ligne traitée
        df.iloc[[index]].to_csv(output_file, index=False, encoding="utf-8", mode='a', header=False)
    
    print(f"✅ Extraction terminée ! Résultats sauvegardés dans {output_file}")

In [ ]:
# Chemins des fichiers
input_gowalla_file = "loc-brightkite_totalCheckins.csv"  # Remplace par ton fichier Gowalla
output_metadata_file = "loc-brightkite_totalCheckins_metadata.csv"
process_gowalla_file(input_gowalla_file, output_metadata_file)

📂 Lecture du fichier Gowalla...


/tmp/ipykernel_5025/2858472314.py:6: DtypeWarning: Columns (5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  last_row = pd.read_csv(output_file).iloc[-1]


🔄 Reprise à partir de l'index 929396
{'name': 'Tama River Bridge, Tama River Cycling Road, Higashi Izumi 4-chome, Komae City, Tokyo, 201-0015, Japan', 'address': 'Tama River Bridge, Tama River Cycling Road, Higashi Izumi 4-chome, Komae City, Tokyo, 201-0015, Japan', 'categories': '[ man_made ; bridge ; man_made ]'}
{'name': 'Tama River Bridge, Tama River Cycling Road, Higashi Izumi 4-chome, Komae City, Tokyo, 201-0015, Japan', 'address': 'Tama River Bridge, Tama River Cycling Road, Higashi Izumi 4-chome, Komae City, Tokyo, 201-0015, Japan', 'categories': '[ man_made ; bridge ; man_made ]'}
{'name': 'Ministry of Economy, Trade and Industry Main Building, Kasumigaseki, Kasumigaseki 1-chome, Kasumigaseki, Chiyoda-ku, Tokyo, 100-0013, Japan', 'address': 'Ministry of Economy, Trade and Industry Main Building, Kasumigaseki, Kasumigaseki 1-chome, Kasumigaseki, Chiyoda-ku, Tokyo, 100-0013, Japan', 'categories': '[ amenity ; public_building ; amenity ]'}
{'name': '2, Ushigome Bridge, Fujimi 2-c